In [1]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Flatten, Concatenate, BatchNormalization, Dropout, MultiHeadAttention, LayerNormalization, Add
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle, resample
from sklearn.metrics import classification_report, confusion_matrix


In [7]:
import sys
sys.path.append('/Users/athenasaghi/VSProjects/CognitiveFatigueDetection/FractalAnalysis')
from CogBeaconDataset import CogBeaconDataset

cogbeacon_root_path = '/Users/athenasaghi/VSProjects/CognitiveFatigueDetection/CogFatigueData/CogBeacon/'
dataset = CogBeaconDataset(cogbeacon_root_path)

In [17]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Flatten, MultiHeadAttention, LayerNormalization, Dropout, Add, BatchNormalization, GlobalAveragePooling1D
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.regularizers import l2
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score, roc_auc_score

# Load Data
df = pd.read_pickle('/Users/athenasaghi/VSProjects/CognitiveFatigueDetection/processed_data_full.pkl')
X_raweeg = np.stack(df['raweeg'].values)
Y_labels = df['label'].values

# Remove class 3
mask = Y_labels != 3
X_raweeg = X_raweeg[mask]
Y_labels = Y_labels[mask]

num_classes = len(np.unique(Y_labels))
Y_labels = to_categorical(Y_labels, num_classes=num_classes)

# Split data
X_train_raweeg, X_test_raweeg, Y_train, Y_test = train_test_split(
    X_raweeg, Y_labels, test_size=0.2, random_state=42, stratify=np.argmax(Y_labels, axis=1)
)

X_train_raweeg, X_val_raweeg, Y_train, Y_val = train_test_split(
    X_train_raweeg, Y_train, test_size=0.1, random_state=42, stratify=np.argmax(Y_train, axis=1)
)

# Standardize Data (only on training set)
scaler = StandardScaler()
X_train_raweeg = scaler.fit_transform(X_train_raweeg.reshape(-1, X_train_raweeg.shape[-1])).reshape(X_train_raweeg.shape)
X_val_raweeg = scaler.transform(X_val_raweeg.reshape(-1, X_val_raweeg.shape[-1])).reshape(X_val_raweeg.shape)
X_test_raweeg = scaler.transform(X_test_raweeg.reshape(-1, X_test_raweeg.shape[-1])).reshape(X_test_raweeg.shape)

# Transformer Model
def build_transformer_model(input_shape, num_classes, num_heads=4, ff_dim=64, dropout_rate=0.2):
    inputs = Input(shape=input_shape)

    # MultiHead Self-Attention
    attn_output = MultiHeadAttention(num_heads=num_heads, key_dim=input_shape[-1])(inputs, inputs)
    attn_output = Dropout(dropout_rate)(attn_output)
    attn_output = Add()([inputs, attn_output])  # Skip connection
    attn_output = LayerNormalization()(attn_output)

    # Feedforward Network
    ff_output = Dense(ff_dim, activation="relu", kernel_regularizer=l2(1e-4))(attn_output)
    ff_output = Dropout(dropout_rate)(ff_output)
    ff_output = Dense(input_shape[-1], activation="relu")(ff_output)
    ff_output = Add()([attn_output, ff_output])  # Skip connection
    ff_output = LayerNormalization()(ff_output)

    # Pooling and Classification
    pooled = GlobalAveragePooling1D()(ff_output)
    x = Dense(128, activation="relu")(pooled)
    x = Dropout(dropout_rate)(x)
    outputs = Dense(num_classes, activation="softmax")(x)

    model = Model(inputs, outputs)
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
                  loss="categorical_crossentropy",
                  metrics=["accuracy", tf.keras.metrics.AUC()])
    return model

# Build and Train Model
input_shape = X_train_raweeg.shape[1:]
model = build_transformer_model(input_shape, num_classes)
model.summary()

early_stopping = EarlyStopping(monitor="val_loss", patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=5)

history = model.fit(
    X_train_raweeg, Y_train,
    validation_data=(X_val_raweeg, Y_val),
    epochs=50, batch_size=32,
    callbacks=[early_stopping, reduce_lr]
)

# Evaluate Model
def evaluate_model(model, X, Y, name):
    Y_pred = model.predict(X)
    accuracy = accuracy_score(np.argmax(Y, axis=1), np.argmax(Y_pred, axis=1))
    auc = roc_auc_score(Y, Y_pred, multi_class="ovr")
    print(f"{name} Accuracy: {accuracy:.4f}, AUC: {auc:.4f}")

evaluate_model(model, X_train_raweeg, Y_train, "Train")
evaluate_model(model, X_val_raweeg, Y_val, "Validation")
evaluate_model(model, X_test_raweeg, Y_test, "Test")


Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2       │ (None, 1854, 4)   │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 1854, 4)   │        308 │ input_layer_2[0]… │
│ (MultiHeadAttentio… │                   │            │ input_layer_2[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_9 (Dropout) │ (None, 1854, 4)   │          0 │ multi_head_atten… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_4 (Add)         │ (None, 1854, 4)   │          0 │ input_layer_2[0]… │
│                     │                   │            │ dropout_9[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 1854, 4)   │          8 │ add_4[0][0]       │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_8 (Dense)     │ (None, 1854, 64)  │        320 │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_10          │ (None, 1854, 64)  │          0 │ dense_8[0][0]     │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_9 (Dense)     │ (None, 1854, 4)   │        260 │ dropout_10[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_5 (Add)         │ (None, 1854, 4)   │          0 │ layer_normalizat… │
│                     │                   │            │ dense_9[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 1854, 4)   │          8 │ add_5[0][0]       │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 4)         │          0 │ layer_normalizat… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_10 (Dense)    │ (None, 128)       │        640 │ global_average_p… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_11          │ (None, 128)       │          0 │ dense_10[0][0]    │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_11 (Dense)    │ (None, 3)         │        387 │ dropout_11[0][0]  │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 1,931 (7.54 KB)

 Trainable params: 1,931 (7.54 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 8s 2s/step - accuracy: 0.3778 - auc_2: 0.5594 - loss: 1.0915 - val_accuracy: 0.1429 - val_auc_2: 0.5867 - val_loss: 1.0726 - learning_rate: 0.0010
Epoch 2/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 6s 1s/step - accuracy: 0.3015 - auc_2: 0.6118 - loss: 1.0666 - val_accuracy: 0.4286 - val_auc_2: 0.6071 - val_loss: 1.0655 - learning_rate: 0.0010
Epoch 3/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 5s 2s/step - accuracy: 0.5528 - auc_2: 0.7018 - loss: 1.0335 - val_accuracy: 0.5714 - val_auc_2: 0.6582 - val_loss: 1.0555 - learning_rate: 0.0010
Epoch 4/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 5s 1s/step - accuracy: 0.5545 - auc_2: 0.7092 - loss: 1.0235 - val_accuracy: 0.5714 - val_auc_2: 0.6582 - val_loss: 1.0486 - learning_rate: 0.0010
Epoch 5/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 5s 1s/step - accuracy: 0.5754 - auc_2: 0.6964 - loss: 1.0209 - val_accuracy: 0.5714 - val_auc_2: 0.6429 - val_loss: 1.0420 - learning_rate: 0.0010
Epoch 6/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 5s 1s/step - accuracy: 0.6430 - auc_2: 0.7747 - l